# Produce tables of stage 2 MWAS results

In [1]:
library(data.table)

In [2]:
getwd()

[1] "/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/scripts"

In [3]:
df <- fread("12-OUT_matched_SNP_meth_cov_outputs.csv")

In [4]:
print(nrow(df))

[1] 16098


In [ ]:
library(data.table)
library(ggplot2)

traits <- c("bp", "mdd", "scz")

for (trait in traits) {
  cat("Processing trait:", trait, "\n")
  df$stage2_path <- gsub(".rds", paste0("_gwas_stat_", trait, "_results.rds"), df$path)

  output_file <- paste0("16a2-OUT_stage2_MWAS_", trait, ".csv")
  header_written <- FALSE
  
  for (i in 1:nrow(df)) {
    if (grepl("empty", df$stage2_path[i])) next
    
    cat("Processing file", i, "of", nrow(df), "\n")
    stage2_in <- readRDS(df$stage2_path[i])
    stage1_in <- readRDS(df$path[i])

    if (length(stage1_in@models) != length(stage2_in@MWASmodels)) {
      stop("Files don't match")
    }

    data_list <- vector("list", length(stage1_in@models))
    for (j in 1:length(stage1_in@models)) {
      this_z <- stage2_in@MWASmodels[[j]]['z']
      this_p <- stage2_in@MWASmodels[[j]]['p']
      this_n <- stage2_in@MWASmodels[[j]]['n']
      this_pos <- stage1_in@models[[j]]@methylationPosition
      this_stats <- stage2_in@summary_stats_path
      this_scaff <- stage1_in@scaffoldIdentifier

      data_list[[j]] <- data.table(
        z = this_z,
        p = this_p,
        n = this_n,
        pos = this_pos,
        stats = this_stats,
        scaff = this_scaff
      )
    }

    combined_data <- rbindlist(data_list, use.names = TRUE, fill = TRUE)

    # Write data incrementally
    if (!header_written) {
      fwrite(combined_data, output_file)
      header_written <- TRUE
    } else {
      fwrite(combined_data, output_file, append = TRUE)
    }
  }
}


In [ ]:
this_p

In [ ]:
this_z

In [ ]:
this_ 